# Photometric Realization from Different Magnitude Error Models

author: John Franklin Crenshaw, Sam Schmidt, Eric Charles, Ziang Yan

last run successfully: August 2, 2023

This notebook demonstrates how to do photometric realization from different magnitude error models. For more completed degrader demo, see `degradation-demo.ipynb`

In [ ]:
import matplotlib.pyplot as plt
from pzflow.examples import get_example_flow
from rail.creation.engines.flowEngine import FlowCreator
from rail.creation.degradation.lsst_error_model import LSSTErrorModel
from rail.core.stage import RailStage


Specify the path to the pretrained 'pzflow' used to generate samples

In [ ]:
import pzflow
import os

flow_file = os.path.join(
    os.path.dirname(pzflow.__file__), "example_files", "example-flow.pzflow.pkl"
)


We'll start by setting up the RAIL data store.  RAIL uses [ceci](https://github.com/LSSTDESC/ceci), which is designed for pipelines rather than interactive notebooks, the data store will work around that and enable us to use data interactively.  See the `rail/examples/goldenspike_examples/goldenspike.ipynb` example notebook for more details on the Data Store.

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True


### "True" Engine

First, let's make an Engine that has no degradation. We can use it to generate a "true" sample, to which we can compare all the degraded samples below.

Note: in this example, we will use a normalizing flow engine from the [pzflow](https://github.com/jfcrenshaw/pzflow) package. However, everything in this notebook is totally agnostic to what the underlying engine is.

The Engine is a type of RailStage object, so we can make one using the `RailStage.make_stage` function for the class of Engine that we want.  We then pass in the configuration parameters as arguments to `make_stage`.

In [ ]:
n_samples = int(1e5)
flowEngine_truth = FlowCreator.make_stage(
    name="truth", model=flow_file, n_samples=n_samples
)


Let's check that the Engine correctly read the underlying PZ Flow object:

In [ ]:
flowEngine_truth.get_data("model")


### Now we invoke the `sample` method to generate some samples

Note that this will return a `DataHandle` object, which can keep both the data itself, and also the path to where the data is written.  When talking to rail stages we can use this as though it were the underlying data and pass it as an argument.  This allows the rail stages to keep track of where their inputs are coming from.

To calculate magnitude error for extended sources, we need the information about major and minor axes of each galaxy. Here we simply generate random values

In [ ]:
samples_truth = flowEngine_truth.sample(n_samples, seed=0)

import numpy as np

samples_truth.data["major"] = np.abs(
    np.random.normal(loc=0.01, scale=0.1, size=n_samples)
)  # add major and minor axes
b_to_a = 1 - 0.5 * np.random.rand(n_samples)
samples_truth.data["minor"] = samples_truth.data["major"] * b_to_a

print(samples_truth())
print("Data was written to ", samples_truth.path)



### LSSTErrorModel

Now, we will demonstrate the `LSSTErrorModel`, which adds photometric errors using a model similar to the model from [Ivezic et al. 2019](https://arxiv.org/abs/0805.2366) (specifically, it uses the model from this paper, without making the high SNR assumption. To restore this assumption and therefore use the exact model from the paper, set `highSNR=True`.)

Let's create an error model with the default settings for point sources:

In [ ]:
errorModel = LSSTErrorModel.make_stage(name="error_model")


For extended sources:

In [ ]:
errorModel_auto = LSSTErrorModel.make_stage(
    name="error_model_auto", extendedType="auto"
)


In [ ]:
errorModel_gaap = LSSTErrorModel.make_stage(
    name="error_model_gaap", extendedType="gaap"
)


Now let's add this error model as a degrader and draw some samples with photometric errors.

In [ ]:
samples_w_errs = errorModel(samples_truth)
samples_w_errs()


In [ ]:
samples_w_errs_gaap = errorModel_gaap(samples_truth)
samples_w_errs_gaap.data


In [ ]:
samples_w_errs_auto = errorModel_auto(samples_truth)
samples_w_errs_auto.data


Notice some of the magnitudes are inf's.
These are non-detections (i.e. the noisy flux was negative).
You can change the nSigma limit for non-detections by setting `sigLim=...`.
For example, if `sigLim=5`, then all fluxes with `SNR<5` are flagged as non-detections.

Let's plot the error as a function of magnitude

In [ ]:
%matplotlib inline

fig, axes_ = plt.subplots(ncols=3, nrows=2, figsize=(15, 9), dpi=100)
axes = axes_.reshape(-1)
for i, band in enumerate("ugrizy"):
    ax = axes[i]
    # pull out the magnitudes and errors
    mags = samples_w_errs.data[band].to_numpy()
    errs = samples_w_errs.data[band + "_err"].to_numpy()
    
    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]
    
    # plot errs vs mags
    #ax.plot(mags, errs, label=band) 
    
    #plt.plot(mags, errs, c='C'+str(i))
    ax.scatter(samples_w_errs_gaap.data[band].to_numpy(),
            samples_w_errs_gaap.data[band + "_err"].to_numpy(),
                s=5, marker='.', color='C0', alpha=0.8, label='GAAP')
    
    ax.plot(mags, errs, color='C3', label='Point source')
    
    
    ax.legend()
    ax.set_xlim(18, 31)
    ax.set_ylim(-0.1, 3.5)
    ax.set(xlabel=band+" Band Magnitude (AB)", ylabel="Error (mags)")


In [ ]:
%matplotlib inline

fig, axes_ = plt.subplots(ncols=3, nrows=2, figsize=(15, 9), dpi=100)
axes = axes_.reshape(-1)
for i, band in enumerate("ugrizy"):
    ax = axes[i]
    # pull out the magnitudes and errors
    mags = samples_w_errs.data[band].to_numpy()
    errs = samples_w_errs.data[band + "_err"].to_numpy()
    
    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]
    
    # plot errs vs mags
    #ax.plot(mags, errs, label=band) 
    
    #plt.plot(mags, errs, c='C'+str(i))
    ax.scatter(samples_w_errs_auto.data[band].to_numpy(),
            samples_w_errs_auto.data[band + "_err"].to_numpy(),
                s=5, marker='.', color='C0', alpha=0.8, label='AUTO')
    
    ax.plot(mags, errs, color='C3', label='Point source')
    
    
    ax.legend()
    ax.set_xlim(18, 31)
    ax.set_ylim(-0.1, 3.5)
    ax.set(xlabel=band+" Band Magnitude (AB)", ylabel="Error (mags)")


You can see that the photometric error increases as magnitude gets dimmer, just like you would expect, and that the extended source errors are greater than the point source errors.
The extended source errors are also scattered, because the galaxies have random sizes.

Also, you can find the GAaP and AUTO magnitude error are scattered due to variable galaxy sizes. Also, you can find that there are gaps between GAAP magnitude error and point souce magnitude error, this is because the additional factors due to aperture sizes have a minimum value of $\sqrt{(\sigma^2+A_{\mathrm{min}})/\sigma^2}$, where $\sigma$ is the width of the beam, $A_{\min}$ is an offset of the aperture sizes (taken to be 0.7 arcmin here).

You can also see that there are *very* faint galaxies in this sample.
That's because, by default, the error model returns magnitudes for all positive fluxes.
If you want these galaxies flagged as non-detections instead, you can set e.g. `sigLim=5`, and everything with `SNR<5` will be flagged as a non-detection.